In [19]:
%%capture
%pip install langchain_community 
%pip install sentence-transformers
%pip install langchain_qdrant

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import pandas as pd
import uuid
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
Qdrant_Demo_GraphRAG = user_secrets.get_secret("Qdrant_Demo_GraphRAG")
write_hf = user_secrets.get_secret("write-hf")

In [4]:
model_embedding = HuggingFaceEmbeddings(
    model_name='hiieu/halong_embedding',
    model_kwargs={'device': device},
    encode_kwargs= {'normalize_embeddings': False}
)

/tmp/ipykernel_23/3560319167.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  model_embedding = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/13.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [5]:
CHUNK_SIZE = 1024
CHUNK_OVERLAP = 256

In [6]:
qa_cntt = pd.read_json('/kaggle/input/dataset-tvpl/ThuVienPhapLuat.QA_CNTT.json').drop('_id', axis=1)
qa_vhxh = pd.read_json('/kaggle/input/dataset-tvpl/ThuVienPhapLuat.QA_VanHoaXaHoi.json').drop('_id', axis=1)
qa_cntt["type"] = 'CNTT'
qa_vhxh["type"] = 'VHXH'
data = pd.concat([qa_cntt, qa_vhxh], axis=0).reset_index(drop=True)
print(data.shape)
data.head(3)

(4009, 7)


,urls,keyword,title,introduction,content,metadata,type
0,https://thuvienphapluat.vn/phap-luat/doanh-ngh...,[Dịch vụ điện toán đám mây],Doanh nghiệp cung cấp dịch vụ điện toán đám mâ...,Doanh nghiệp cung cấp dịch vụ điện toán đám mâ...,[{'sub_title': 'Doanh nghiệp cung cấp dịch vụ ...,"{'time_published': '01:40', 'date_published': ...",CNTT
1,https://thuvienphapluat.vn/phap-luat/phuong-ti...,[Tài khoản định danh điện tử],Phương tiện xác thực là gì? Tài khoản định dan...,Theo quy định thì phương tiện xác thực là gì? ...,"[{'sub_title': 'Phương tiện xác thực là gì?', ...","{'time_published': '23:23', 'date_published': ...",CNTT
2,https://thuvienphapluat.vn/phap-luat/07-thong-...,[Zalo BHXH Việt Nam],07 thông tin được cung cấp trên Zalo BHXH Việt...,07 thông tin được cung cấp trên Zalo BHXH Việt...,[{'sub_title': '07 thông tin được cung cấp trê...,"{'time_published': '20:42', 'date_published': ...",CNTT


In [7]:
def flatten(xss):
    return [x for xs in xss for x in xs]

In [8]:
def prepare_chunks_not_split(data):
    chunks = []
    metadata = data.get("metadata", {})
    base_metadata = {
        "url": data["urls"],
        "date_published": metadata.get("date_published"),
        "author": metadata.get("author"),
    }

    # Chia nhỏ nội dung từng sub_content
    for section in data["content"]:
        sub_title = section["sub_title"]
        sub_content = section["sub_content"]

        chunk_metadata = {
            "sub_title": sub_title,
            **base_metadata,
        }
        chunks.append({
            "id": str(uuid.uuid4()),
            "text": sub_content,
            "metadata": chunk_metadata
        })
    return chunks

prepared_chunks = [prepare_chunks_not_split(sample) for sample in data.iloc]
len(prepared_chunks)

4009

In [9]:
chunks = flatten(prepared_chunks)
len(chunks)

12188

In [10]:
from torch.utils.data import DataLoader

batch_size = 256
loader = DataLoader(chunks, batch_size=batch_size, shuffle=False)

In [11]:
embeddings = []
for batch in loader:
    texts = [chunk for chunk in batch["text"]]
    batch_embeddings = model_embedding.embed_documents(texts)
    embeddings.extend(batch_embeddings)

# Gắn embeddings
for i, chunk in enumerate(chunks):
    chunk["embedding"] = embeddings[i]


In [24]:
df = pd.DataFrame(data=chunks)
df.head()

,id,text,metadata,embedding
0,bd81d41f-de51-49c4-8c95-1d9e7e800c49,Căn cứ theo điểm c khoản 1 Điều 29 Luật Viễn t...,{'sub_title': 'Doanh nghiệp cung cấp dịch vụ đ...,"[0.01842622458934784, -0.03174770623445511, 0...."
1,d0083cb8-c873-4d86-a636-57a1ca533427,Căn cứ theo điểm a khoản 2 Điều 29 Luật Viễn t...,{'sub_title': 'Doanh nghiệp cung cấp dịch vụ đ...,"[0.024296294897794724, -0.06644633412361145, 0..."
2,7eea9044-3ae1-4407-b420-3ccf9385a1d4,Căn cứ theo khoản 4 Điều 4 Luật Viễn thông 202...,{'sub_title': 'Nhà nước có chính sách thúc đẩy...,"[0.04203903302550316, -0.024394724518060684, -..."
3,2ed363fb-e752-451c-a0eb-17e6c2818923,Căn cứ theo khoản 8 Điều 3 Nghị định 69/2024/N...,"{'sub_title': 'Phương tiện xác thực là gì?', '...","[-0.044873014092445374, 0.0033915129024535418,..."
4,afa9d800-5ddb-4483-a2d5-8e353171e5e2,Căn cứ theo điểm a khoản 1 Điều 20 Nghị định 6...,{'sub_title': 'Tài khoản định danh điện tử mức...,"[-0.07656639814376831, -0.0073141371831297874,..."


In [27]:
from langchain.schema import Document
# Chuyển đổi DataFrame thành danh sách Document
chunks = [
    Document(
        page_content=row["text"],
        metadata=row["metadata"]
    )
    for _, row in df.iterrows()
]

In [38]:
import qdrant_client
client = qdrant_client.QdrantClient(   
    url="https://21a75178-7457-4e63-974b-666f8174af84.us-west-2-0.aws.cloud.qdrant.io:6333", 
    api_key=Qdrant_Demo_GraphRAG,
)

In [47]:
from qdrant_client.models import VectorParams, Distance, CollectionConfig

client.create_collection(
    collection_name="qa_tvpl",
    vectors_config=VectorParams(
        size=768,  # Kích thước vector của embedding (chỉnh sửa theo embedding của bạn)
        distance=Distance.COSINE,  # Khoảng cách cosine là phổ biến cho các embedding dense
    ),
)
print("Collection đã được tạo thành công.")

Collection đã được tạo thành công.


In [48]:
vector_store = QdrantVectorStore(client=client, collection_name="qa_tvpl",embedding=model_embedding)

In [52]:
from qdrant_client.models import PointStruct

# Đặt kích thước batch (ví dụ: 1000 điểm mỗi batch)
batch_size = 1000

# Tạo các điểm dữ liệu
points = [
    PointStruct(
        id=row["id"],  # ID của mỗi điểm dữ liệu
        vector=row["embedding"],  # Embedding vector đã tính toán
        payload={
            "text": row["text"],  # Dữ liệu văn bản
            "metadata": row["metadata"]  # Dữ liệu metadata (nếu có)
        }
    )
    for _, row in df.iterrows()
]

# Chia dữ liệu thành các batch
for i in range(0, len(points), batch_size):
    batch_points = points[i:i + batch_size]
    
    # Thêm vào Qdrant
    vector_store.client.upsert(
        collection_name="qa_tvpl",
        points=batch_points
    )

print("Dữ liệu đã được lưu thành công vào Qdrant.")


Dữ liệu đã được lưu thành công vào Qdrant.
